In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from xgboost import XGBClassifier

In [34]:
# data = pd.read_csv("csv/walk.csv",index_col=False)
# X = data[["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang",]]
#         # "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
#         # "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
# # X = data.drop(['gender', "id"], axis=1)

# encode = {'male': 0, 'female': 1}
# y = data['gender'].map(encode)
# X  = preprocessing.scale(X)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [54]:
data = pd.read_csv("csv/walk.csv",index_col=False)
encode = {'male': 0, 'female': 1}

X = data.loc[data['id'].isin([1,2])]
X_train = X [["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang",
        "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
        "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
#X_train = X.drop(['id','gender'],axis=1)
X_train = preprocessing.scale(X_train)
y_train = X['gender'].map(encode)

X2 = data.loc[data['id'].isin([3,4])]
X_test = X2 [["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang",
        "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
        "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
#X_test = X2.drop(['id','gender'],axis=1)
X_test = preprocessing.scale(X_test)
y_test = X2['gender'].map(encode)

In [57]:
clf = XGBClassifier(reg_lambda = 1,
 reg_alpha = 0,
 n_estimators = 500,
 learning_rate = 0.05,
 gamma = 0.5,
 booster = 'gbtree',
 base_score = 0.2)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [58]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 0.7903011781736298
[[ 9634  3295]
 [ 1920 10020]]
              precision    recall  f1-score   support

           0       0.83      0.75      0.79     12929
           1       0.75      0.84      0.79     11940

    accuracy                           0.79     24869
   macro avg       0.79      0.79      0.79     24869
weighted avg       0.79      0.79      0.79     24869



In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

param = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01,0.05,0.1],
    'booster': ['gbtree', 'gblinear'],
    'gamma': [0, 0.5, 1],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0.5, 1, 5],
    'base_score': [0.2, 0.5, 1]
}

In [52]:
rnd_search = RandomizedSearchCV(XGBClassifier(), param_distributions=param,n_jobs=-1, verbose = 10, n_iter =10)
rnd_search.fit(X_train,y_train)
rnd_search.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\jocel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jocel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jocel\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\Users\jocel\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py", line 15

0.9196879257301193

In [53]:
rnd_search.best_params_

{'reg_lambda': 1,
 'reg_alpha': 0,
 'n_estimators': 500,
 'learning_rate': 0.05,
 'gamma': 0.5,
 'booster': 'gbtree',
 'base_score': 0.2}